In [1]:
from PIL import Image, ImageSequence, ImageFont, ImageDraw
from PIL.TiffTags import TAGS
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import pickle
import pandas as pd
import cv2
from skimage.filters import threshold_otsu
import copy
from skimage.measure import label
from skimage import morphology as morph
import zut
import CombineColors as CC
from ipywidgets import widgets
from matplotlib.cm import get_cmap
from IPython.display import display
from ipyevents import Event 
import io

def nonzero(im):
    r = np.ravel(im)
    return r[r>0]

dataset = 'JIE_melanoma'
project_base = f'/Users/liuzi/Documents/IMC/{dataset}/'
os.chdir(project_base)
if not os.path.isdir('processing'):
    os.mkdir('processing')

rawdata = pickle.load(open(f'data/raw_1.p','rb'))
fn_ls = list(rawdata.keys())

fn = fn_ls[0]
markers = list(rawdata[fn]['im'].keys())
print('all markers:')
print(markers)
markers_used = [
    'CD3', 'Ki67','DNA1','CD68','Melanoma'
]
markers_used = markers_used + [
    'aSMA', # epithelial (Smooth muscle alpha-actin)
    'B7-H4', # TAM, co-inhibitory factor inhibiting CD4+ and CD8+ T-cell proliferation
    'CD4', 'CD8a', # T sub
    'CD14', # DC
    'CD11b', # myeloid
    'CD11c', # DC
    'CD15', # neutrophil
    'CD56', # NK
    'CD19-CD20', # B cells
    # # 'CD16', 
    'CD31', # endothelial
    'CD33', # myeloid
    'CD45', 'CD45RO','CD45RA', # pan immune
    'CD74', # chaperone of HLA-DR
    # # 'CD80', 
    'Collagen-I', 
    'Epcam+E-Cadherin', # melanoma
    'HLA-ABC', 
    'HLA-DR', # APC
    # # 'IL-1b', ?
    # # 'Krt8-18', ? 
    # # 'MITF', # melanoma
    # # 'MX1', 
    # # 'PD-1', 
    # # 'PD-L1', 
    # # 'Tryptase', 
    # # 'Vimentin'

]
markers_used = np.unique([m for m in markers_used if m in markers])
print('\nMarkers used:')
print(sorted(markers_used))
print(len(markers_used))

fn_ls_good = ['roi' + str(i) for i in [4, 6, 10, 12, 15, 17, 20]]
fn_ls_fine = ['roi' + str(i) for i in [19, 22, 24, 27]]
fn_ls = fn_ls_good + fn_ls_fine

data = pickle.load(open('data/data.p', 'rb'))
cell_df = pickle.load(open('data/cell_df.p','rb'))

# log normalization and remove lowest bin in histogram
for fn in fn_ls:
    for m in markers_used:
        im = np.log(rawdata[fn]['im'][m] + 1)
        h = np.histogram(nonzero(im),bins=64)
        im [im < h[1][np.argmax(h[0])+1]] = 0
        data[fn]['im'][m] = im

all markers:
['131Xe', '134Xe', '173Yb', '209Bi', '80ArAr', 'B7-H4', 'CD11b', 'CD11c', 'CD14', 'CD15', 'CD16', 'CD19-CD20', 'CD3', 'CD31', 'CD33', 'CD4', 'CD45', 'CD45RA', 'CD45RO', 'CD56', 'CD68', 'CD74', 'CD80', 'CD8a', 'Collagen-I', 'DNA1', 'DNA3', 'Epcam+E-Cadherin', 'HLA-ABC', 'HLA-DR', 'IL-1b', 'Ki67', 'Krt8-18', 'MITF', 'MX1', 'Melanoma', 'None', 'PD-1', 'PD-L1', 'Tryptase', 'Vimentin', 'aSMA']

Markers used:
['B7-H4', 'CD11b', 'CD11c', 'CD14', 'CD15', 'CD19-CD20', 'CD3', 'CD31', 'CD33', 'CD4', 'CD45', 'CD45RA', 'CD45RO', 'CD56', 'CD68', 'CD74', 'CD8a', 'Collagen-I', 'DNA1', 'Epcam+E-Cadherin', 'HLA-ABC', 'HLA-DR', 'Ki67', 'Melanoma', 'aSMA']
25


In [2]:


def readchannel(data, fn, m, p=99):
    ''''Read a channel and cap at p percentile'''
    im = data[fn]['im'][m]
    im = np.where(im > np.percentile(nonzero(im) , p), np.percentile(nonzero(im) , p), im)
    return im

def uint(x):
    return (np.array(x)*255).astype('uint8')

def addHist(Im, mat, t1=None, t2 =None):
    '''Add a tiny pixel histogram of array mat at the cornor of an image Im. '''
    Im = Im.copy()
    nn_nonzero = nonzero(mat)
    plt.rc('xtick', labelsize=10) 
    fig = plt.figure(facecolor=(1,1,1), figsize=(1.5,1))
    plt.hist(nn_nonzero, bins = 40)
    if t1:
        plt.axvline(t1, c = (1, 0, 0))
    if t2:
        plt.axvline(t2, c = (0, 1, 0))

    plt.tick_params(axis='both', which='both', top=False,
                        bottom=True, left=False, right=False,
                        labelbottom=True, labelleft=False)
    plt.tight_layout()
    fig.canvas.draw()
    Image.frombytes('RGB', 
    fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    fig_hist = Image.frombytes('RGB', 
        fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    plt.close()
    Im.paste(fig_hist, (0, int(Im.size[1]*0.92 - fig_hist.size[1]*0.9)))
    return Im

def plotcellmean(im, cellmasks,  cmap='turbo'):
    p = 99
    cm = get_cmap(cmap)
    cell_dict = {}

    cellnum = np.max(cellmasks) 
    im_c = np.zeros_like(im)
    for cell_id in np.arange(1, cellnum+1):
        cellmask_cur=1*(cellmasks==cell_id)
        cellarea = np.sum(cellmask_cur)  
        cell_dict[cell_id] = np.sum(cellmask_cur*im)/cellarea
    cellvalues = np.array(list(cell_dict.values()))
    cellvalues = np.where(cellvalues > np.percentile(nonzero(cellvalues) , p), np.percentile(nonzero(cellvalues) , p), cellvalues)
    
    im = np.zeros((im.shape[0], im.shape[1], 3))
    
    for cell_id in np.arange(1, cellnum+1):
        im_c[cellmasks==cell_id] = cellvalues[cell_id-1]/max(np.max(cellvalues), 0.00001)

    c_df = pd.DataFrame(im_c)
    c_df = c_df.stack().reset_index()

    for j in [0,1,2]:
        im[c_df.iloc[:,0], c_df.iloc[:,1], j] = cm(c_df.iloc[:,2])[:,j]
    Im = Image.fromarray(uint(im))

    cm = get_cmap(cmap)
    Im = CC.add_legend(Im, {f"{c*np.max(cellvalues):.1f}":tuple(uint(cm(c)[0:3])) for c in np.array([0.25, 0.5, 0.75, 1.0])})
    Im = addHist(Im, cellvalues)
    return Im

def plotpixelvalue(data, fn, m, t, cmax, showdna = True, showfiltered = True):
    im = readchannel(data, fn, m)
    im_b = readchannel(data, fn, 'DNA1')
    im_out = im.copy()
    im_out[im > t] =0
    im_r = im.copy()
    im_r[im_r < t] = 0
    im_r = im_r/max(cmax, 0.001)
    im_r[im_r > 1] = 1
    pd = {f'{m}_pass':im_r}
    cd = {f'{m}_pass':'red'}
    if showdna:
        pd['DNA1'] = im_b
        cd['DNA1'] = 'blue'
    if showfiltered:
        pd[f'{m}_out'] = im_out
        cd[f'{m}_out'] = 'green'
    Im = CC.CombineColor(
        pd,
        cd,
        mode='add',
        palette={'red':(255,0,0), 'grey':(122,122,122), 'blue':(0,0,255), 'green':(0,255,0)}
    )
    Im = addHist(Im, im, t1 = t, t2 = cmax)
    return Im

In [3]:
# # Segmentation

# from deepcell.applications import Mesmer
# import skimage.morphology as morph
# app = Mesmer()

# dna_marker = 'DNA1'
# mem_markers = ['CD45','HLA-ABC','HLA-DR','Melanoma']

# for fn in fn_ls:

#     data[fn]['mask'] = {}
#     im_dna = readchannel(data, fn, dna_marker)
#     im_mem = np.max(np.dstack([readchannel(data, fn, m) for m in mem_markers]), axis=2)


#     im_mesmer = np.dstack([im_dna, im_mem])
#     im_mesmer = im_mesmer[np.newaxis,:,:]
#     cmask = app.predict(im_mesmer)

#     print(f'{fn}: {np.max(cmask)} cells.')
#     cellmask = cmask[0,:,:,0]
#     kernel = np.array([[0,1,0],[1,1,1],[0,1,0]])

#     cell_n = np.max(cellmask)
#     inner_mask = np.zeros(np.shape(cellmask))
#     bound_mask = np.zeros(np.shape(cellmask))
#     for cell_id in range(1, cell_n):
#         cellmask_cur = cellmask == cell_id
#         innermask_cur = morph.binary_erosion(cellmask_cur, kernel)
#         boundmask_cur = cellmask_cur ^ innermask_cur
#         inner_mask = inner_mask + cell_id * innermask_cur
#         bound_mask = bound_mask + cell_id * boundmask_cur
#     data[fn]['mask']['cellmask'] = cellmask
#     data[fn]['mask']['inner'] = inner_mask
#     data[fn]['mask']['bound'] = bound_mask

In [4]:
def investigate(data, fn, m, t, cmax, showdna, showfiltered):
    fig = plt.figure(facecolor=(1,1,1), figsize=(30,18))
    plt.subplot(121)
    plt.imshow(plotpixelvalue(data, fn, m, t, cmax, showdna, showfiltered))
    plt.title('Pixels')
    zut.axisoff()
    
    im = readchannel(data, fn, m)
    im[im<t] = 0
    im[im>cmax]=cmax
    plt.subplot(122)
    plt.imshow(plotcellmean(im, data[fn]['mask']['cellmask'],  cmap='inferno'))
    plt.title('Cells')
    zut.axisoff()

    plt.suptitle(f"{fn}, {m}, t={t:.3f}, cmax={cmax:.3f}")

    plt.tight_layout()
    return fig

def showhist(data, fn, m, t, cmax):
    mat = readchannel(data, fn, m)
    nn_nonzero = nonzero(mat)
    plt.rc('xtick', labelsize=10) 
    fig = plt.figure(facecolor=(1,1,1), figsize=(5,1))
    plt.hist(nn_nonzero, bins = 40)
    if t:
        plt.axvline(t, c = (1, 0, 0))
    if cmax:
        plt.axvline(cmax, c = (0, 1, 0))

    plt.tick_params(axis='both', which='both', top=False,
                        bottom=True, left=False, right=False,
                        labelbottom=True, labelleft=False)
    plt.tight_layout()

In [5]:
fndrop = widgets.Dropdown(options = fn_ls)
mdrop = widgets.Dropdown(options = markers_used)
tslide = widgets.FloatSlider(label='t',value=0, min=0, max=4, step=0.01, continuous_update=False)
cmaxslide = widgets.FloatSlider(label='cmax', value=4, min=0, max=4, step=0.01, continuous_update=False)
dnacheck = widgets.Checkbox(True, description='show DNA channel')
filteredcheck = widgets.Checkbox(True, description='show filtered-out pixels')


def changemax(*args):
    tslide.max = np.max(readchannel(data,fndrop.value, mdrop.value))
    tslide.value = 0.0
    cmaxslide.max = np.max(readchannel(data,fndrop.value, mdrop.value))
    cmaxslide.value = cmaxslide.max

fndrop.observe(changemax)
mdrop.observe(changemax)
hb1 = widgets.HBox((fndrop, mdrop))
hb2 = widgets.HBox((tslide, cmaxslide))
vb = widgets.VBox((hb1, hb2))
fndrop.value = 'roi4'
mdrop.value = 'CD3'
# w = widgets.interactive(investigate, 
#     data=widgets.fixed(data), fn=fndrop, m=mdrop, t=tslide, cmax=cmaxslide, __manual=True)

/Users/liuzi/opt/anaconda3/lib/python3.8/site-packages/traitlets/traitlets.py:586: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


In [6]:
widgets.interactive_output(investigate, 
    {'data':widgets.fixed(data),'fn':fndrop, 'm':mdrop, 't':tslide, 'cmax':cmaxslide, 'showdna':dnacheck, 'showfiltered':filteredcheck})


Output()

In [7]:
{
    "tags": [
        "hide_input",
    ]
}
display(vb, widgets.HBox((dnacheck, filteredcheck)))
# widgets.interactive(showhist, data=widgets.fixed(data), fn=fndrop, m=mdrop, t=tslide, cmax=cmaxslide)
widgets.interactive_output(showhist, 
    {'data':widgets.fixed(data),'fn':fndrop, 'm':mdrop, 't':tslide, 'cmax':cmaxslide})


Output()

0.0

In [65]:
# to save the figure

fn = fndrop.value
m = mdrop.value
t = tslide.value
cmax = cmaxslide.value
fig = investigate(data, fn, m, t, cmax, dnacheck.value, filteredcheck.value)
plt.savefig(f"{fn}_{m}_{t:.2}_{cmax:.2}.png")
plt.close()

In [13]:
import io

def findcellid(cellmask, x, y):
    return cellmask[x, y]

def showcellloc(boundmask, cell_id):
    bound_all = np.where(data[fn]['mask']['bound']>0, 1, 0) 
    if cell_id > 0:
        bound_cur = np.where(data[fn]['mask']['bound']==cell_id, 1, 0) 
        Im = CC.CombineColor({'all':bound_all, 'cur':bound_cur}, ['w', 'r'],addlegend=False )
    else:
        Im = CC.CombineColor({'all':bound_all}, ['w'],addlegend=False )
    return Im

def Im2Bytes(Im):
    img_byte_arr = io.BytesIO()
    Im.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

In [ ]:

def cell_level_class(data, cell_df, fn, level, m = None, label = None,cmap='inferno'):
    cm = get_cmap(cmap)
    df = cell_df.loc[cell_df['Image']==fn, ['Cell',level ]]
    cellmask = data[fn]['mask']['cellmask']
    im = np.zeros((cellmask.shape[0], cellmask.shape[1], 3))
    legend_dict = {}

    if m:
        df[f'{m}_ave'] = cell_df.loc[cell_df['Image']==fn, f'{m}_ave']

        im_c = np.zeros_like(cellmask, dtype=float)
        for cell_id in df['Cell']:
            im_c[cellmask==cell_id] = df.loc[df['Cell']==cell_id, f'{m}_ave']/max(np.max(df[f'{m}_ave']), 0.00001)
    
        c_df = pd.DataFrame(im_c)
        c_df = c_df.stack().reset_index()

        legend_dict[f'{m}_low'] = tuple(uint(cm(0.35)[0:3]))
        legend_dict[f'{m}_high'] = tuple(uint(cm(0.85)[0:3]))

        for j in [0,1,2]:
            im[c_df.iloc[:,0], c_df.iloc[:,1], j] = cm(c_df.iloc[:,2])[:,j]
    im = uint(im)

    if label:
        boundcolor = (0, 255, 255)
        boundmask = data[fn]['mask']['bound']
        cell_toshow = df.loc[df[level]==label, 'Cell']
        if len(cell_toshow)>0:
            im_bound = np.any(np.dstack([boundmask == i for i in cell_toshow]), axis=2)
            for j in [0,1,2]:
                im[im_bound,j] = boundcolor[j]
        legend_dict[f'{level}={label}:{len(cell_toshow)}'] = boundcolor

    Im = Image.fromarray(im)
    Im = CC.add_legend(Im, legend_dict)
    # plt.figure(facecolor=(1,1,1), figsize=(7,7))
    # plt.imshow(Im)
    # plt.title('Cell values')
    # zut.axisoff()

In [15]:
fn = 'roi4'

boundmask = data[fn]['mask']['bound']
cellmask = data[fn]['mask']['cellmask']


w_im = widgets.Image(value=Im2Bytes(showcellloc(boundmask, 0)), format='png')
w_im.layout.max_width = '60%'
w_im.layout.height = 'auto'


im_events = Event()
im_events.source = w_im
im_events.watched_events = ['click']

no_drag = Event(source=w_im, watched_events=['dragstart'], prevent_default_action = True)

coordinates = widgets.Label('Click an image to see the click coordinates here')

vbox = widgets.VBox([w_im, coordinates])

def update_coords(event):
    cell_id = findcellid(data[fn]['mask']['cellmask'], event['dataY'], event['dataX'])
    coordinates.value = 'Clicked at ({}, {}). Cell_id = {}.'.format(event['dataX'], event['dataY'], cell_id)
    w_im.value = Im2Bytes(showcellloc(boundmask, cell_id))

im_events.on_dom_event(update_coords)

display(vbox)